In [3]:
import requests

In [4]:
#scrape the webpages

In [5]:
fbs_data_p1_url = 'https://www.ncaa.com/stats/football/fbs/current/individual/469'

In [6]:
fbs_data_p2_url = 'https://www.ncaa.com/stats/football/fbs/current/individual/469/p2'

In [7]:
fbs_data_p3_url = 'https://www.ncaa.com/stats/football/fbs/current/individual/469/p3'

In [8]:
fbs_data_p4_url = 'https://www.ncaa.com/stats/football/fbs/current/individual/469/p4'

In [9]:
#name response variables for each webpage

In [10]:
response1 = requests.get(fbs_data_p1_url)

In [11]:
response2 = requests.get(fbs_data_p2_url)

In [12]:
response3 = requests.get(fbs_data_p3_url)

In [13]:
response4 = requests.get(fbs_data_p4_url)

In [14]:
#display all status codes so we know the responses are valid

In [15]:
response1.status_code

200

In [16]:
response2.status_code

200

In [17]:
response3.status_code

200

In [18]:
response4.status_code

200

In [19]:
#i'd like to loop through the status codes and only print something if the code isn't 200-299

In [20]:
#name page content variables that point to the text

In [21]:
fbs_pc1 = response1.text

In [22]:
fbs_pc2 = response2.text

In [23]:
fbs_pc3 = response3.text

In [24]:
fbs_pc4 = response4.text

In [25]:
#idk if i need this part, but I'm leaving it in for now

In [26]:
#with open('webpage.html', 'w') as f:
    #f.write(page_contents)

In [27]:
#get beautifulsoup

In [28]:
!pip install beautifulsoup4 --upgrade --quiet

In [29]:
from bs4 import BeautifulSoup

In [30]:
#name 4 bs docs that carry the page contents

In [31]:
fbs_doc1 = BeautifulSoup(fbs_pc1, 'html.parser')

In [32]:
fbs_doc2 = BeautifulSoup(fbs_pc2, 'html.parser')

In [33]:
fbs_doc3 = BeautifulSoup(fbs_pc3, 'html.parser')

In [34]:
fbs_doc4 = BeautifulSoup(fbs_pc4, 'html.parser')

In [35]:
#get the table row tags from each doc

In [37]:
tr_tags1 = fbs_doc1.find_all('tr')

In [38]:
tr_tags2 = fbs_doc2.find_all('tr')

In [39]:
tr_tags3 = fbs_doc3.find_all('tr')

In [40]:
tr_tags4 = fbs_doc4.find_all('tr')

In [39]:
#get ast

In [40]:
import ast

In [41]:
#get pandas

In [42]:
import pandas as pd

In [43]:
#name df columns

In [44]:
columns = ['Rank', 'Name', 'Team', 'Cl', 'Position','G', 'Rush', 'Rush TD', 'Rush Yds']

In [45]:
#make a function to get the string list into a workable format

In [46]:
def pull_out_row(tr_tag_row):
    new_info = str(tr_tag_row)
    new_info = new_info.replace('"', '')
    new_info = new_info.replace('<td>','"')
    new_info = new_info.replace('</td>','",')
    new_info = new_info.replace('<tr>', '[')
    new_info = new_info.replace('</tr>', ']')
    new_info = new_info.replace(',\n]', ']')
    new_info = ast.literal_eval(new_info)
    
    new_info = pd.DataFrame(new_info).T
    
    return new_info

In [47]:
#create a function to create a table from all of our formatted rows

In [48]:
def parse_table(table, columns):
    table = table[1:]
    
    master_df = pull_out_row(table.pop(0))

    for i in table:
        temp = pull_out_row(i)
        master_df = master_df.append(temp, ignore_index = True)

    master_df.columns = columns

    return master_df

In [53]:
df1 = parse_table(tr_tags1, columns)

In [54]:
df2 = parse_table(tr_tags2, columns)

In [55]:
df3 = parse_table(tr_tags3, columns)

In [56]:
df4 = parse_table(tr_tags4, columns)

In [58]:
bigtable_fbs = df1.append(df2, ignore_index = True)

In [60]:
bigtable_fbs = bigtable.append(df3, ignore_index = True)

In [61]:
bigtable_fbs = bigtable.append(df4, ignore_index = True)

In [62]:
bigtable_fbs

,Rank,Name,Team,Cl,Position,G,Rush,Rush TD,Rush Yds
0,1,Lew Nichols,<img src=https://i.turner.ncaa.com/sites/defau...,Fr.,RB,12,311,15,1710
1,2,Kenneth Walker III,<img src=https://i.turner.ncaa.com/sites/defau...,Jr.,RB,12,263,18,1636
2,3,Tyler Badie,<img src=https://i.turner.ncaa.com/sites/defau...,Sr.,RB,12,268,14,1604
3,4,Tyler Allgeier,<img src=https://i.turner.ncaa.com/sites/defau...,So.,RB,13,276,23,1601
4,5,Sean Tucker,<img src=https://i.turner.ncaa.com/sites/defau...,So.,RB,12,246,12,1496
...,...,...,...,...,...,...,...,...,...
195,196,Tyhier Tyler,<img src=https://i.turner.ncaa.com/sites/defau...,Jr.,QB,11,125,7,486
196,197,Rodney Hammond Jr.,<img src=https://i.turner.ncaa.com/sites/defau...,Fr.,RB,11,98,5,484
197,198,Justice Ellison,<img src=https://i.turner.ncaa.com/sites/defau...,Fr.,RB,10,98,6,482
198,-,Levi Williams,<img src=https://i.turner.ncaa.com/sites/defau...,So.,QB,9,72,5,482


In [63]:
bigtable_fbs.to_csv('FBS_bigTable.csv')